# Analyis of grid hyperparameter search

In [ ]:
import json
import pathlib
import pandas as pd
import plotly.express as px

import vaep.io
import vaep.pandas
import vaep.utils

pd.options.display.max_columns = 30
pd.options.display.multi_sparse = False

### Papermill parameters

papermill parameters:

In [ ]:
metrics_json:str = "path/to/all_metrics.json" # file path to metrics json
configs_json:str = "path/to/all_configs.json" # file path to configs json ("meta data")

## Load metrics

In [ ]:
path_metrics_json = pathlib.Path(metrics_json)
path_configs_json = pathlib.Path(configs_json)
FOLDER = path_metrics_json.parent

metrics_dict = vaep.io.load_json(path_metrics_json)
configs_dict = vaep.io.load_json(path_configs_json)

Random sample metric schema (all should be the same)

In [ ]:
key_sampled = vaep.utils.sample_iterable(metrics_dict, 1)[0]
key_map = vaep.pandas.key_map(metrics_dict[key_sampled])
key_map

Metrics a `pandas.DataFrame`:

In [ ]:
metrics = {}
for k, run_metrics in metrics_dict.items():
    metrics[k] = vaep.pandas.flatten_dict_of_dicts(run_metrics)

metrics_dict_multikey = metrics

metrics = pd.DataFrame.from_dict(metrics, orient='index')
metrics.columns.names = ['subset','data_split', 'model', 'metric_name']
metrics

In [ ]:
metrics['NA interpolated']

In [ ]:
# import collections
# subset_keys = ['_'.join(x.split()) for x in metrics.columns.levels[0]]
# MetricSubsets = collections.namedtuple("MetricSubsets", subset_keys)
# metric_subsets = MetricSubsets(*(metrics[k] for k in metrics.columns.levels[0]))

In [ ]:
sort_by = 'MSE'
metric_columns = ['MSE', 'MAE']
subset = metrics.columns.levels[0][0]
print(f"{subset = }")

## Metadata

Experiment metadata from configs

In [ ]:
meta = pd.read_json(path_configs_json).T
meta['hidden_layers'] = meta['hidden_layers'].apply(tuple) # make list a tuple
meta['n_hidden_layers'] = meta.hidden_layers.apply(len)
meta.head()

Batch size for collab models depends on a factor (as the data in long format has roughly  N samples * M features entries).

In [ ]:
# ToDo: Load from config
meta['bs_collab'] = meta['batch_size'] * 8
meta.head()

### Plot Top 10 for collab validation data
- options see [2Dline plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D)

In [ ]:
_  = metrics[subset]["valid_collab"]['collab'].sort_values(
    'MSE').iloc[:10, :-1].plot(rot=90, 
                          x_compat=True, 
                          xticks=list(range(10)),
                          marker='o',
                          linestyle='',
                          title='Top 10 results for hyperparameters'
                         )

## Colorcoded metrics

- can be one of the [matplotlib color maps](https://matplotlib.org/stable/tutorials/colors/colormaps.html), which also have reversed version indicated by `*_r`

``` python
['Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r']
```

In [ ]:
cmap='cividis_r'

In [ ]:
metrics = metrics.set_index(pd.MultiIndex.from_frame(meta[['latent_dim', 'n_hidden_layers', 'batch_size']])).sort_index()
metrics_styled = metrics.style.background_gradient(cmap)
metrics_styled

In [ ]:
metrics_styled.to_excel(FOLDER/ 'metrics_styled.xlsx')

In [ ]:
for k in metrics.columns.levels[0][::-1]:
    print("\n"+"*"*10, f"Subset: {k}\n")
    display(metrics[k].style.background_gradient(cmap))

### Plot Top 10 for collab validation data with numeric x-axis labels

In [ ]:
_ = metrics[subset]["valid_collab"]['collab'].sort_values(
    'MSE').iloc[:10,:-1].plot(rot=90,
                          x_compat=False,
                          xticks=list(range(10)),
                          marker='o',
                          linestyle=''
                          )

## Collection of Performance plots 

- similar to hyperparameter performance plots in Tensorboard

In [ ]:
metrics = metrics.reset_index()

In [ ]:
metrics.iloc[:, :7].head(3)

### Parallel coordinates

- similar to Tensorboard visualization of a set of hyperparameters

In [ ]:
pos_metric = 3
metric_sel = metrics.iloc[:,[0,1,2,pos_metric]]
title = metrics.columns[pos_metric]
metric_sel.columns = [' '.join( x[0].split('_')) for x in metric_sel.columns[:-1]] + [title[-1]]
metric_sel.head(2)

In [ ]:
fig = px.parallel_categories(metric_sel, dimensions=metric_sel.columns[:-1],
                color="MSE", color_continuous_scale=px.colors.sequential.Inferno,
                title=' '.join(title[0].split('_'))
)
fig.show()

### Plotting without Multi-Index

In [ ]:
metrics = pd.json_normalize([{'index': k, **d} for k,d in metrics_dict.items()], meta='index', sep= ' ')
metrics = metrics.set_index('index')
metrics = meta.join(metrics)
metrics

In [ ]:
labels_dict = {"NA not interpolated valid_collab collab MSE": 'MSE',
               'bs_collab': 'bs',
               'n_hidden_layers': "No. of hidden layers",
               'latent_dim': 'hidden layer dimension'}

#### Single model metric

In [ ]:
col = "NA not interpolated valid_collab collab MSE"
fig = px.scatter(metrics,
                 x="latent_dim",
                 y=col,
                 # color="hidden layers", # needs data in long format
                 facet_row="bs_collab",
                 facet_col="n_hidden_layers",
                 title='Performance on validation data for collaborative filtering model',
                 labels=labels_dict
                )
fig.show()

### Plotting from long format

To use colors meaningfully, the long format of the data is needed.

In [ ]:
metrics_long = pd.DataFrame.from_dict(metrics_dict_multikey, orient='index')
metrics_long.columns.names = ['subset', 'data_split', 'model', 'metric_name']
metrics_N = metrics_long.loc[:, pd.IndexSlice[:,:,:, 'N']]
metrics_N.columns = [' '.join([*x[:2], x[3]]) for x in metrics_N.columns] #0, 1, 3 column index
metrics_N = metrics_N.loc[:, ~metrics_N.T.duplicated()] # remove duplicated
metrics_long=metrics_long.loc[:, pd.IndexSlice[:,:,:, metric_columns]]
# all or subset:
metrics_N = metrics_N.set_index(pd.MultiIndex.from_frame(meta))
metrics_long = metrics_long.set_index(pd.MultiIndex.from_frame(metrics_N.reset_index()))
# pd.MultiIndex.from_frame(metrics_N.reset_index())
# # pd.MultiIndex.from_frame(meta[['latent_dim', 'n_hidden_layers', 'hidden_layers', 'batch_size', 'batch_size_collab', ]])
metrics_long = metrics_long.stack(metrics_long.columns.names)
metrics_long = metrics_long.to_frame('metric_value').reset_index().set_index('data_split')
metrics_long

In [ ]:
sorted(metrics_N.columns, key=lambda x: x[::-1])

#### All model metrics for one subset of data

In [ ]:
dataset = 'test_fake_na'


def get_plotly_figure(dataset):
    fig = px.scatter(metrics_long.loc[dataset],
                     x="latent_dim",
                     y='metric_value',
                     color="model",
                     facet_row="metric_name",
                     facet_col="subset",
                     hover_data=['n_hidden_layers', 'hidden_layers',
                                 'batch_size', 'batch_size_collab',
                                 'n_params_ae', 'n_params_collab', # n_params_vae'], # along other variables
                                 'subset', f'NA not interpolated {dataset} N', f'NA interpolated {dataset} N'],
                     title=f'Performance on {dataset.replace("_", " ")} data',
                     labels={**labels_dict,
                               "metric_value": '', 'metric_name': 'metric'}
                     )
    return fig


fig = get_plotly_figure(dataset)
fig.show()

In [ ]:
dataset = 'valid_fake_na'
fig = get_plotly_figure(dataset)
fig.show()

In [ ]:
dataset = 'valid_collab'
fig = get_plotly_figure(dataset)
fig.show()